### 【 D0123_work_이준기 】

[ 2026-01-23] 과제

NYC Taxi Dataset

① 요금(Fare Amount) 회귀 예측  	 => MAE

② 장거리 Trip 분류 			 => F1

③ 공항 Trip 여부 분류 			 => F1

④ Trip Duration(이동 시간) 회귀 예측 => MAE

### 순서도 (계획)
1. 과제 번호 선택 (3번)
2. 데이터 확인 (컬럼 확인, 어떤 컬럼 사용할지 등)
3. 4개 데이터 전부 불러와서 전처리 진행
4. 공항 Trip 라벨 생성 (공항 좌표 정의)
5. Feature 선택 및 데이터셋 구성
6. 모델 설계 및 학습
7. 모델 평가
8. 결과 분석 및 해석
9. 결론 및 향후 개선 방향

In [10]:
## -------------------------------------------------------------------------------
## 모듈 로딩
## -------------------------------------------------------------------------------
import pandas as pd

In [11]:
## -------------------------------------------------------------------------------
## 데이터 확인
## -------------------------------------------------------------------------------
DATA_FILE_TEST = '../D0123/Data/yellow_tripdata_2015-01.csv'

df_T = pd.read_csv(DATA_FILE_TEST)

df_T.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30


In [12]:
df_T.shape

(12748986, 19)

- 총 컬럼 개수 19개
- 각 컬럼 의미 및 선택 여부
1. VendorID : 택시 회사 ID
2. tpep_pickup_datetime : 승차 시각
3. tpep_dropoff_datetime : 하차 시각
4. passenger_count : 승객 수
5. trip_distance : 이동 거리
6. pickup_longitude : 승차 경도
7. pickup_latitude : 승차 위도
8. RateCodeID : 요금 정책 코드
9. store_and_fwd_flag : 서버 전송 지연 여부
10. dropoff_longitude : 하차 경도
11. dropoff_latitude : 하차 위도
12. payment_type : 결제 방식
13. fare_amount : 기본 요금
14. extra : 추가 요금
15. mta_tax : 세금
16. tip_amount : 팁
17. tolls_amount : 톨게이트 요금
18. improvement_surcharge : 추가 요금
19. total_amount : 총 요금

- 사용할 컬럼
1. passenger_count : 승객 수
2. trip_distance : 이동 거리
3. pickup_longitude : 승차 경도
4. pickup_latitude : 승차 위도
5. dropoff_longitude : 하차 경도
6. dropoff_latitude : 하차 위도

-> 승차 시각과 하차 시각은 가공해서 탑승 시간으로 새로운 컬럼 생성 예정

In [ ]:
## -------------------------------------------------------------------------------
## 전처리 함수 생성
## -------------------------------------------------------------------------------
def preprocessing(df):

    ## 결측치 제거
    df = df.dropna()

    ## 선택 컬럼만 추출하기
    use_cols = [
    'passenger_count',
    'trip_distance',
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude'
    ]
    df = df[use_cols]

    ## 승객 수가 0인 이상치 제거


    ## 이동 거리가 0인 이상치 제거


    ## 

    return df

In [15]:
df_Tc = df_T.copy()

In [19]:
df_Tc = preprocessing(df_Tc)
df_Tc.head()

,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,1,1.59,-73.993896,40.750111,-73.974785,40.750618
1,1,3.30,-74.001648,40.724243,-73.994415,40.759109
2,1,1.80,-73.963341,40.802788,-73.951820,40.824413
3,1,0.50,-74.009087,40.713818,-74.004326,40.719986
4,1,3.00,-73.971176,40.762428,-74.004181,40.742653
